In [1]:
import OffsetArrays
using Plots
using Revise
using StatsBase
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using Distributions
srcpath = "./src"
include("$srcpath/bp.jl") #julia code which contains the bp update
include("$srcpath/post_infer.jl") #definition of the main functions and the types
include("$srcpath/observables.jl") # julia code for estimating observables

avg_ninf (generic function with 1 method)

In [11]:
λp = 1.0# planted infection rate
λi = λp #inferred infection rate

T = 8 # discrete time: number of time-steps

γp = 0.1 #0.019 # planted autoinfection probability
γi = γp # inferred autoinfection probability

N = 5000; #population size
@show 2/sqrt(N)

dilution = 0.0 #dilution of observations. dil=0 means everybody observed once
fr = 0.0; #noise in the observation. 

degree_dist = Dirac(3) #the distribution of the degree in the graph. Dirac means random regular.

2 / sqrt(N) = 0.0282842712474619


Dirac{Int64}(value=3)

In [55]:
M_zero = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [56]:
#@time sweep!(M_zero)

In [57]:
F_zero,it_zero = pop_dynamics(M_zero, tot_iterations = 20);

#1.iter 2.err 3.F
0	-1	-0.2018627106821623
1	-1	-0.391096365020729
2	-1	-0.39102746431940466
3	-1	-0.391014407261827
4	-1	-0.39101439547166444
5	-1	-0.39101439547165867
6	-1	-0.39101439547165867
7	-1	-0.39101439547165867
8	-1	-0.39101439547165867
9	-1	-0.39101439547165867
10	-1	-0.39101439547165867
11	3.920262300842004e-6	-0.39101439547165867
12	0.0	-0.39101439547165867
13	0.0	-0.39101439547165867
14	0.0	-0.3910143954716579
15	0.0	-0.39101439547165867
16	0.0	-0.39101439547165867
17	0.0	-0.39101439547165867
18	0.0	-0.39101439547165867
19	0.0	-0.39101439547165867


In [58]:
include("$srcpath/post_infer.jl") #definition of the main functions and the types

δλ (generic function with 1 method)

In [59]:
F_zero,it_zero = pop_dynamics(M_zero, tot_iterations = 1);

#1.iter 2.err 3.F
0	-1	-0.2004861810204755


In [60]:
l=rand(1:N)
M_zero.belief[:,:,l]

10×10 OffsetArray(::Matrix{Float64}, 0:9, 0:9) with eltype Float64 with indices 0:9×0:9:
 0.0  0.0100478    0.0318314    0.0294296    …  0.000256018  0.0
 0.0  0.0245066    0.0776368    0.0717788       0.000624427  0.0
 0.0  0.0274769    0.087047     0.0804789       0.000700113  0.0
 0.0  0.0193088    0.0611701    0.0565546       0.000491987  0.0
 0.0  0.010455     0.0331213    0.0306221       0.000266392  0.0
 0.0  0.00519425   0.0164554    0.0152138    …  0.000132349  0.0
 0.0  0.00230205   0.0072929    0.00674262      5.86563e-5   0.0
 0.0  0.000829055  0.00262645   0.00242827      2.11671e-5   0.0
 0.0  0.000167973  0.000532139  0.000491987     4.71146e-6   0.0
 0.0  0.0          0.0          0.0             0.0          9.25462e-8

In [64]:
xi0, sij, sji, d, oi, sympt, ci, ti_obs=rand_disorder(M_zero, degree_dist)
neighs = rand(1:N,d)
z_i=calculate_belief!(M_zero,l,neighs,xi0,oi,sympt,ci,ti_obs)

2.1865474526405927e-6

In [65]:
log(z_i)

-13.033186763772065

In [62]:
M_zero.belief[:,:,l]

10×10 OffsetArray(::Matrix{Float64}, 0:9, 0:9) with eltype Float64 with indices 0:9×0:9:
 0.0  0.0  0.100035     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.243961     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.30747      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.250145     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0905509    0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.00778379   0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  5.44935e-5   0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  9.65858e-9   0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.64591e-14  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [63]:
sum(M_zero.belief[:,:,l])

0.9999999999999999

In [ ]:
#M_zero.mom1μ
#sum(M_zero.mom1μ)./length(M_zero.mom1μ)

In [ ]:
#marg_zero = M_zero.belief |> real;
#marg2D_zero = reshape(sum(marg_zero,dims=3) ./ N, T+2,T+2)
#ensAUC_zero = avgAUC(marg_zero,M_zero.obs_list,count_obs=true);

In [5]:
M = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [6]:
F,it = pop_dynamics_stab(M, tot_iterations = 100, nbstab=40);

#1.iter 2.err 3.F 4. Δ
0	-1	0.15499788473197143	-1.0
1	-1	0.04331987229660154	-1.0
2	-1	0.20711930418033037	-1.0
3	-1	0.41249110709306436	-1.0
4	-1	0.5653547701957508	-1.0


LoadError: InterruptException:

In [ ]:
@time sweep_stab!(M, 3, 4)

In [ ]:
test = rand(-1:2:1, T+2, 2, T+2, 3, N).*0.00001;
mean(test), mean(M.μ), mean(M.mom1μ)

In [ ]:
extrema(test), extrema(M.μ), extrema(M.mom1μ)

In [ ]:
std(test), std(M.μ), std(M.mom1μ)

In [ ]:
marg = M.belief |> real;
marg2D = reshape(sum(marg,dims=3) ./ N, T+2,T+2)
ensAUC = avgAUC(marg,M.obs_list,count_obs=true); #set false to see the AUC only on NON observed indiv.

In [ ]:
sum(abs.(marg2D.-marg2D_zero))/length(marg2D)

In [ ]:
scatter(ensAUC)
scatter!(ensAUC_zero)

In [ ]:
pheat = heatmap((marg2D),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred",title="Average 2D marginal")
pheat_stab = heatmap((marg2D_zero.-marg2D),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred")
plot(pheat,pheat_stab)

In [ ]:
scatter(F)
scatter!(F_zero)
hline!([0])